In [10]:
import datajoint as dj
dj.conn()
import csv

In [2]:
from workflow_session.pipeline import lab, subject, session

In [6]:
lab.Lab()

lab Abbreviated lab name,lab_name full lab name,institution,address,"time_zone UTC offset suggested e.g., UTC+1"


In [7]:
subject.Subject()

subject,sex,subject_birth_date,subject_description


In [8]:
session.Session()

subject,session_datetime


In [103]:
lab.Lab.delete()
lab.Project.delete()
lab.Protocol.delete()

Deleting 2 rows from `neuro_lab`.`#location`
Deleting 2 rows from `neuro_lab`.`#lab`


Commit deletes? [yes, No]:  yes


Deletes committed.
Deleting 2 rows from `neuro_lab`.`#project`


Commit deletes? [yes, No]:  yes


Deletes committed.
Deleting 2 rows from `neuro_lab`.`#protocol`


Commit deletes? [yes, No]:  yes


Deletes committed.


2

In [142]:
import element_data_loader.utils
from workflow_session.paths import *

In [144]:
def get_root_data_dir():
    root_data_dirs = dj.config.get('custom', {}).get('root_data_dir', None)
    return root_data_dirs if root_data_dirs else None

In [159]:
lab.Project.Publication()

project,publication


In [160]:
def ingest_lab(lab_csv_path='./user_data/lab/labs.csv',
                project_csv_path='./user_data/lab/projects.csv',
                pubs_csv_path='./user_data/lab/publications.csv',
                keyw_csv_path='./user_data/lab/keywords.csv',
                protocol_csv_path='./user_data/lab/protocols.csv'
                ):
    # -------------- Insert new "Lab" --------------
    with open(lab_csv_path, newline= '') as f:
        input_labs = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert {len(input_labs)} entry(s) into lab tables ----')
    lab.Lab.insert(input_labs, skip_duplicates=True, ignore_extra_fields=True)
    lab.Location.insert(input_labs, skip_duplicates=True, ignore_extra_fields=True)
    # -------------- Insert new "Project" --------------
    with open(project_csv_path, newline= '') as f:
        input_projs = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert {len(input_projs)} entry(s) into project table ----')
    lab.Project.insert(input_projs, skip_duplicates=True, ignore_extra_fields=True)
    # -------------- Insert publications + keywords --------------
    with open(pubs_csv_path, newline= '') as f:
        input_pubs = list(csv.DictReader(f, delimiter=','))
    with open(keyw_csv_path, newline= '') as f:
        input_keyw = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert entry(s) into publication/keyword tables ----')
    lab.Project.Publication.insert(input_pubs, skip_duplicates=True, ignore_extra_fields=True)
    lab.Project.Keywords.insert(input_keyw, skip_duplicates=True, ignore_extra_fields=True)

    # -------------- Insert new "Protocol" --------------
    with open(protocol_csv_path, newline= '') as f:
        input_prots = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert {len(input_prots)} entry(s) into protocol tables ----')
    lab.Protocol.insert(input_prots, skip_duplicates=True, ignore_extra_fields=True)
    lab.ProtocolType.insert(input_prots, skip_duplicates=True, ignore_extra_fields=True)
def ingest_subjects(subject_csv_path='./user_data/animal/subjects.csv'):
    # -------------- Insert new "Subject" --------------
    with open(subject_csv_path, newline= '') as f:
        input_subjects = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert {len(input_subjects)} entry(s) into subject tables ----')
    subject.Subject.insert(input_subjects, skip_duplicates=True, ignore_extra_fields=True)
    subject.SubjectDeath.insert(input_subjects, skip_duplicates=True, ignore_extra_fields=True)
    subject.SubjectCullMethod.insert(input_subjects, skip_duplicates=True, ignore_extra_fields=True)
    ## Skipped allele info
def ingest_sessions(session_csv_path='./user_data/session/sessions.csv'):
    with open(session_csv_path, newline= '') as f:
        input_sessions = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Insert {len(input_sessions)} entry(s) into session.Session ----')
    session.Session.insert(input_sessions, skip_duplicates=True, ignore_extra_fields=True)
    session.SessionDirectory.insert(input_sessions, skip_duplicates=True, ignore_extra_fields=True)
    session.SessionNote.insert(input_sessions, skip_duplicates=True, ignore_extra_fields=True)
ingest_lab(); ingest_subjects();ingest_sessions()


---- Insert 2 entry(s) into lab tables ----

---- Insert 2 entry(s) into project table ----

---- Insert entry(s) into publication/keyword tables ----

---- Insert 2 entry(s) into protocol tables ----

---- Insert 2 entry(s) into subject tables ----

---- Insert 2 entry(s) into session.Session ----


In [77]:
lab.Lab & 'lab="LabA"'

lab Abbreviated lab name,lab_name full lab name,institution,address,"time_zone UTC offset suggested e.g., UTC+1"
LabA,The Example Lab,Example Uni,"221B Baker St,London NW1 6XE,UK",UTC+0


In [78]:
lab.Project & 'project="ProjA"'

project,publication
ProjA,arXiv:1807.11104
ProjA,arXiv:1807.11104v1


In [88]:
lab.Protocol() & 'protocol="ProtA"'

protocol,protocol_type,protocol_description
ProtA,IRB expedited review,Protocol for managing data ingestion


In [105]:
lab_info = (lab.Lab & 'lab="LabA"').fetch1()
proj_info = (lab.Project & 'project="ProjA"').fetch1()
prot_info = (lab.Protocol() & 'protocol="ProtA"').fetch1()

In [131]:
lab.Lab & 'lab="None"'

lab Abbreviated lab name,lab_name full lab name,institution,address,"time_zone UTC offset suggested e.g., UTC+1"


In [92]:
from pynwb import NWBFile

In [178]:
project_key = 'project="ProjA"'
proj_info = (lab.Project & project_key).fetch1()
proj_keyw = (lab.Project.Keywords() & project_key).fetch('keyword').tolist()
proj_pubs = (lab.Project.Publication() & project_key).fetch('publication').tolist()
NWBFile(
    # data_collection='',
    experiment_description=proj_info['project_description'],
    keywords=proj_keyw,
    notes=prot_info['protocol_description'],
    pharmacology=proj_info['pharmacology'],
    related_publications=proj_pubs,
    slices=proj_info['slices'],
    source_script=proj_info['repositoryurl'],
    file_name=proj_info['repositoryname'],
    stimulus=proj_info['stimulus'],
    surgery=proj_info['surgery']
)

TypeError: NWBFile.__init__: missing argument 'session_description', missing argument 'identifier', missing argument 'session_start_time', incorrect type for 'stimulus' (got 'str', expected 'list or tuple'), unrecognized argument: 'file_name'

In [130]:
if tmp := proj_info['virus']: virus=proj_info['virus']

SyntaxError: invalid syntax (255707863.py, line 1)

In [ ]:
 NWBFile(identifier='_'.join(session_identifier.values()),
                   session_description=session_info['session_note'] if session_info['session_note'] else '',
                   session_start_time=session_info['session_datetime'],
                   file_create_date=datetime.now(tzlocal()),
                   experimenter=list(experimenters)